In [1]:
import music21 as m21
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import glob

# Step 1: Load and preprocess the dataset
def load_midi_files(midi_dir):
    notes = []
    for file in glob.glob(midi_dir + "/*.midi"):
        midi = m21.converter.parse(file)
        parts = m21.instrument.partitionByInstrument(midi)
        if parts:  # File has instrument parts
            notes_to_parse = parts.parts[0].recurse()
        else:  # File has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, m21.note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, m21.chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

def prepare_sequences(notes, n_vocab, sequence_length=100):
    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    input_sequences = []
    output_notes = []

    for i in range(0, len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        input_sequences.append([note_to_int[char] for char in sequence_in])
        output_notes.append(note_to_int[sequence_out])

    n_patterns = len(input_sequences)

    X = np.reshape(input_sequences, (n_patterns, sequence_length, 1))
    X = X / float(n_vocab)
    y = to_categorical(output_notes, num_classes=n_vocab)

    return X, y, note_to_int

# Step 2: Build the model
def create_network(input_shape, n_vocab):
    model = Sequential()
    model.add(LSTM(512, input_shape=input_shape, recurrent_dropout=0.3, return_sequences=True))
    model.add(LSTM(512, recurrent_dropout=0.3))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

# Step 3: Train the model
def train_network(model, X, y, epochs=10, batch_size=64):  # Reduced epochs to 10
    model.fit(X, y, epochs=epochs, batch_size=batch_size)

# Step 4: Generate music
def generate_music(model, notes, n_vocab, note_to_int, sequence_length=100, num_generate=500):
    start = np.random.randint(0, len(notes) - sequence_length - 1)
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = [note_to_int[char] for char in notes[start:start + sequence_length]]
    generated_notes = []

    for i in range(num_generate):
        input_pattern = np.reshape(pattern, (1, len(pattern), 1))
        input_pattern = input_pattern / float(n_vocab)

        prediction = model.predict(input_pattern, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        generated_notes.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return generated_notes

def create_midi(generated_notes, output_file='output.mid'):
    offset = 0
    output_notes = []

    for pattern in generated_notes:
        if ('.' in pattern) or pattern.isdigit():
            chord_notes = pattern.split('.')
            notes = [m21.note.Note(int(n)) for n in chord_notes]
            chord = m21.chord.Chord(notes)
            chord.offset = offset
            output_notes.append(chord)
        else:
            note = m21.note.Note(pattern)
            note.offset = offset
            output_notes.append(note)
        offset += 0.5

    midi_stream = m21.stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_file)

# Main function to run the steps
def main():
    midi_files_dir = "C:\MUSIC GENERATION WITH AI\Musi Generation project\Pop_Music_Midi"  # Add your MIDI files directory here
    notes = load_midi_files(midi_files_dir)
    n_vocab = len(set(notes))

    X, y, note_to_int = prepare_sequences(notes, n_vocab)
    model = create_network((X.shape[1], X.shape[2]), n_vocab)

    train_network(model, X, y)

    generated_notes = generate_music(model, notes, n_vocab, note_to_int)
    create_midi(generated_notes)

if __name__ == "__main__":
    main()


Epoch 1/10
69/69 [==============================] - 1890s 27s/step - loss: 3.0071
Epoch 2/10
69/69 [==============================] - 1515s 22s/step - loss: 2.8814
Epoch 3/10
62/69 [=========================>....] - ETA: 2:51 - loss: 2.8567